In [2]:
%run SetUpParams.ipynb

2024-08-11 15:16:48.445073
2004-08-11 15:16:48.445073


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [3]:
import pandas as pd 
import numpy as np

### Create dataframe to feed into model

In [55]:
final_df = pd.DataFrame()
# Next Day Open Price - Current Day Open Price
final_df[f'{target_stock}_change'] = target_stock_data['Open'].shift(-1) - target_stock_data['Open']
final_df[f'{target_stock}_lag'] = final_df[f'{target_stock}_change'].shift(1)
final_df['Close'] = target_stock_data['Close']
final_df['Price'] = target_stock_data['Open']
final_df.reset_index(inplace=True)
print(final_df)

           Date  ^GSPC_change   ^GSPC_lag        Close        Price
0    2004-08-11     -3.250000         NaN  1075.790039  1079.040039
1    2004-08-12    -12.560059   -3.250000  1063.229980  1075.790039
2    2004-08-13      1.570068  -12.560059  1064.800049  1063.229980
3    2004-08-16     14.539917    1.570068  1079.339966  1064.800049
4    2004-08-17      2.369995   14.539917  1081.709961  1079.339966
...         ...           ...         ...          ...          ...
5029 2024-08-05     55.279785 -225.489746  5186.330078  5151.140137
5030 2024-08-06     86.709961   55.279785  5240.029785  5206.419922
5031 2024-08-07    -40.560059   86.709961  5199.500000  5293.129883
5032 2024-08-08     62.090332  -40.560059  5319.310059  5252.569824
5033 2024-08-09           NaN   62.090332  5344.160156  5314.660156

[5034 rows x 5 columns]


#### Add European and US Stock Market Price Changes to final_df
Current Day Open Price - Previous Day Open Price

In [56]:
# EU Stock Market has already opened
for ticker in US_EU_market_dict.keys():
        data = US_EU_market_dict[ticker]
        # Current Day Open Price - Previous Day Open Price
        temp = (data['Open'] - data['Open'].shift(1)).reset_index()
        temp.rename(columns={'Open':ticker}, inplace=True)
        final_df = final_df.merge(temp, on='Date', how='left')

print(final_df)

           Date  ^GSPC_change   ^GSPC_lag        Close        Price  \
0    2004-08-11     -3.250000         NaN  1075.790039  1079.040039   
1    2004-08-12    -12.560059   -3.250000  1063.229980  1075.790039   
2    2004-08-13      1.570068  -12.560059  1064.800049  1063.229980   
3    2004-08-16     14.539917    1.570068  1079.339966  1064.800049   
4    2004-08-17      2.369995   14.539917  1081.709961  1079.339966   
...         ...           ...         ...          ...          ...   
5029 2024-08-05     55.279785 -225.489746  5186.330078  5151.140137   
5030 2024-08-06     86.709961   55.279785  5240.029785  5206.419922   
5031 2024-08-07    -40.560059   86.709961  5199.500000  5293.129883   
5032 2024-08-08     62.090332  -40.560059  5319.310059  5252.569824   
5033 2024-08-09           NaN   62.090332  5344.160156  5314.660156   

           ^GSPC        ^IXIC         ^DJI       ^FCHI      ^GDAXI  
0            NaN          NaN          NaN         NaN         NaN  
1      -3

#### Add Asian Stock Market Price Changes to final_df
Close Price - Open Price

In [57]:
# Asian Stock Market has already closed; one day ahead of US Stock Market
for ticker in asia_market_dict.keys():
        data = asia_market_dict[ticker]
        # Close Price - Open Price
        temp = (data['Close'] - data['Open']).reset_index()
        temp.rename(columns={0:ticker}, inplace=True)
        final_df = final_df.merge(temp, on='Date', how='left')

print(final_df)

           Date  ^GSPC_change   ^GSPC_lag        Close        Price  \
0    2004-08-11     -3.250000         NaN  1075.790039  1079.040039   
1    2004-08-12    -12.560059   -3.250000  1063.229980  1075.790039   
2    2004-08-13      1.570068  -12.560059  1064.800049  1063.229980   
3    2004-08-16     14.539917    1.570068  1079.339966  1064.800049   
4    2004-08-17      2.369995   14.539917  1081.709961  1079.339966   
...         ...           ...         ...          ...          ...   
5029 2024-08-05     55.279785 -225.489746  5186.330078  5151.140137   
5030 2024-08-06     86.709961   55.279785  5240.029785  5206.419922   
5031 2024-08-07    -40.560059   86.709961  5199.500000  5293.129883   
5032 2024-08-08     62.090332  -40.560059  5319.310059  5252.569824   
5033 2024-08-09           NaN   62.090332  5344.160156  5314.660156   

           ^GSPC        ^IXIC         ^DJI       ^FCHI      ^GDAXI  \
0            NaN          NaN          NaN         NaN         NaN   
1      

In [58]:
# Different markets has different holidays
# Forward fill null values
final_df.fillna(method='ffill', inplace=True)
final_df.isna().sum()
final_df.dropna(inplace=True)

C:\Users\chuac\AppData\Local\Temp\ipykernel_30968\3476143031.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  final_df.fillna(method='ffill', inplace=True)


Date            0
^GSPC_change    0
^GSPC_lag       1
Close           0
Price           0
^GSPC           1
^IXIC           1
^DJI            1
^FCHI           1
^GDAXI          1
^AORD           0
^HSI            0
^N225           0
dtype: int64